In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from urllib.request import urlopen
import json
from scipy.interpolate import CubicSpline

In [2]:
with open("../.fmp_api.key", "r") as f:
    api_key = f.read().strip()

url = f"https://financialmodelingprep.com/stable/treasury-rates?apikey={api_key}"
response = urlopen(url)
par_rates = pd.DataFrame(json.loads(response.read().decode("utf-8")))

In [3]:
par_rates

,date,month1,month2,month3,month6,year1,year2,year3,year5,year7,year10,year20,year30
0,2025-04-24,4.34,4.37,4.32,4.22,3.97,3.77,3.80,3.91,4.11,4.32,4.79,4.77
1,2025-04-23,4.33,4.34,4.33,4.22,4.01,3.81,3.87,4.00,4.20,4.40,4.86,4.83
2,2025-04-22,4.33,4.35,4.33,4.21,3.98,3.76,3.82,3.98,4.19,4.41,4.90,4.88
3,2025-04-21,4.35,4.37,4.34,4.21,3.95,3.75,3.77,3.97,4.19,4.42,4.94,4.91
4,2025-04-17,4.36,4.38,4.34,4.22,3.99,3.81,3.82,3.95,4.13,4.34,4.82,4.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2025-01-31,4.37,4.37,4.31,4.28,4.17,4.22,4.27,4.36,4.47,4.58,4.88,4.83
58,2025-01-30,4.37,4.38,4.30,4.27,4.16,4.18,4.24,4.31,4.41,4.52,4.81,4.76
59,2025-01-29,4.43,4.34,4.31,4.27,4.17,4.21,4.27,4.35,4.44,4.55,4.85,4.79
60,2025-01-28,4.44,4.35,4.31,4.26,4.14,4.19,4.25,4.33,4.43,4.55,4.84,4.78


In [4]:
tenors = [(t, float(t.replace("month", "").replace("year","")) / \
           (12 if "month" in t else 1)) for t in par_rates.columns[1:]]

xt = np.concatenate((np.array([t[1] for t in tenors[:4]]), np.arange(1, 30.5, 0.5)))

tenors, xt

([('month1', 0.08333333333333333),
  ('month2', 0.16666666666666666),
  ('month3', 0.25),
  ('month6', 0.5),
  ('year1', 1.0),
  ('year2', 2.0),
  ('year3', 3.0),
  ('year5', 5.0),
  ('year7', 7.0),
  ('year10', 10.0),
  ('year20', 20.0),
  ('year30', 30.0)],
 array([ 0.08333333,  0.16666667,  0.25      ,  0.5       ,  1.        ,
         1.5       ,  2.        ,  2.5       ,  3.        ,  3.5       ,
         4.        ,  4.5       ,  5.        ,  5.5       ,  6.        ,
         6.5       ,  7.        ,  7.5       ,  8.        ,  8.5       ,
         9.        ,  9.5       , 10.        , 10.5       , 11.        ,
        11.5       , 12.        , 12.5       , 13.        , 13.5       ,
        14.        , 14.5       , 15.        , 15.5       , 16.        ,
        16.5       , 17.        , 17.5       , 18.        , 18.5       ,
        19.        , 19.5       , 20.        , 20.5       , 21.        ,
        21.5       , 22.        , 22.5       , 23.        , 23.5       ,
        24

In [5]:
def interpolate_curve(tenors, rates, xt):
    spline = CubicSpline(
        [t[1] for t in tenors], rates, bc_type="natural"
    )
    yt = spline(xt)
    return (pd.DataFrame(
            {
                "xt": xt,
                "par_rate": yt,
            }
        ).assign(key_rate = lambda df_: [1 if df_.xt.iloc[i] in [t[1] for t in tenors] else 0 for i in range(df_.shape[0])])
    )

interpolated_curve = interpolate_curve(tenors, par_rates.iloc[0, 1:], xt)

In [6]:
interpolated_curve

,xt,par_rate,key_rate
0,0.083333,4.340000,1
1,0.166667,4.370000,1
2,0.250000,4.320000,1
3,0.500000,4.220000,1
4,1.000000,3.970000,1
...,...,...,...
58,28.000000,4.798089,0
59,28.500000,4.791396,0
60,29.000000,4.784421,0
61,29.500000,4.777257,0


In [ ]:
interpolated_curve.where(lambda df_: df_.xt.ge(0.5) & df_.x)

,xt,par_rate,key_rate
0,0.083333,4.340000,1
1,0.166667,4.370000,1
2,0.250000,4.320000,1
3,0.500000,4.220000,1
4,1.000000,3.970000,1
...,...,...,...
58,28.000000,4.798089,0
59,28.500000,4.791396,0
60,29.000000,4.784421,0
61,29.500000,4.777257,0


In [26]:
def bootstrapSpotRates(par_rates):
    """
    Compute the spot rate for a given maturity i using the par rates.
    """
    spot_rates = []
    subyear_counter = 0
    for i in range(par_rates.shape[0]):
        if par_rates.xt.iloc[i] < 1:
            spot_rates.append(par_rates.par_rate.iloc[i])
            subyear_counter += 1
        else:
            fv = 100
            c = par_rates.par_rate.iloc[i] / 2.
            xt = np.arange(0.5, par_rates.xt.iloc[i] + 0.5, 0.5)
            rt = spot_rates[subyear_counter - 1:]
            df = np.sum([1. / (1 + rt[j] / 200.) ** (2 * xt[j]) for j in range(len(rt))])
            spot_rates.append((((fv + c) / (fv - c * df)) ** (1. / len(xt))- 1) * 200)
    return pd.Series(spot_rates, index=par_rates.index)
    
interpolated_curve["spot_rate"] = bootstrapSpotRates(interpolated_curve)

In [28]:
interpolated_curve.tail(20)

,xt,par_rate,key_rate,spot_rate
43,20.5,4.800621,0,5.000014
44,21.0,4.809454,0,5.011236
45,21.5,4.816593,0,5.019659
46,22.0,4.822133,0,5.025380
47,22.5,4.826167,0,5.028504
48,23.0,4.828790,0,5.029145
49,23.5,4.830095,0,5.027425
50,24.0,4.830177,0,5.023475
51,24.5,4.829130,0,5.017431
52,25.0,4.827048,0,5.009438


In [34]:
curve_10y = interpolated_curve.iloc[3:23].loc[:, ["xt", "par_rate", "spot_rate"]].copy()

def pv_bond(coupon_rate, spot_rates, face_value=100):
    """
    Compute the present value of a bond given its coupon rate, maturity, and face value.
    """
    coupon = coupon_rate / 200 * face_value
    cash_flows = np.array([coupon] * (spot_rates.size -1) + [face_value + coupon])
    discount_factors = (1 + curve_10y.spot_rate / 200) ** (-2 * curve_10y.xt)
    pv = np.sum(cash_flows * discount_factors)
    return pv

pv_bond(4.32, curve_10y.spot_rate, 1000)

np.float64(1000.0000000000007)

In [118]:
def pv_bond_ytm(coupon_rate, ytm, n_maturity, face_value=100):
    """
    Compute the present value of a bond given its coupon rate, yield to maturity, and maturity.
    """
    coupon = coupon_rate / 200 * face_value
    cash_flows = np.array([coupon] * (n_maturity * 2 - 1) + [face_value + coupon])
    discount_factors = (1 + ytm / 200.) ** (-2 * np.arange(0.5, n_maturity + 0.5, 0.5))
    pv = np.sum(cash_flows * discount_factors)
    return pv

def analytic_derivative(bond_price, coupon_rate, ytm, n_maturity, face_value=100):
    """
    Compute the derivative of the present value of a bond with respect to its coupon rate.
    """
    coupon = coupon_rate / 200. * face_value
    cash_flows = np.array([coupon * i for i in np.arange(0.5, n_maturity, 0.5)] + [n_maturity * (face_value + coupon)])
    discount_factors = (1 + ytm / 200.) ** (-(2 * np.arange(0.5, n_maturity + 0.5, 0.5) + 1))
    pv = np.sum(cash_flows * discount_factors)
    
    return pv

def forward_difference(bond_price, coupon_rate, ytm, n_maturity, face_value=100, delta=1e-5):
    """
    Compute the derivative of the present value of a bond with respect to its coupon rate using finite differences.
    """
    f1 = bond_price - pv_bond_ytm(coupon_rate, ytm + delta * 100, n_maturity, face_value)
    f2 = bond_price - pv_bond_ytm(coupon_rate, ytm, n_maturity, face_value)
    return (f1 - f2) / delta

def backward_difference(bond_price, coupon_rate, ytm, n_maturity, face_value=100, delta=1e-5):
    """
    Compute the derivative of the present value of a bond with respect to its coupon rate using finite differences.
    """
    f1 = bond_price - pv_bond_ytm(coupon_rate, ytm, n_maturity, face_value)
    f2 = bond_price - pv_bond_ytm(coupon_rate, ytm - delta * 100, n_maturity, face_value)
    return (f1 - f2) / delta

def central_difference(bond_price, coupon_rate, ytm, n_maturity, face_value=100, delta=1e-5):
    """
    Compute the derivative of the present value of a bond with respect to its coupon rate using finite differences.
    """
    f1 = bond_price - pv_bond_ytm(coupon_rate, ytm + delta * 100, n_maturity, face_value)
    f2 = bond_price - pv_bond_ytm(coupon_rate, ytm - delta * 100, n_maturity, face_value)
    return (f1 - f2) / (2 * delta)


def yield_to_maturity(bond_price, coupon_rate, n_maturity, face_value=100, derivative_func = analytic_derivative, tolerance=1e-6, max_iterations=10000, *args, **kwargs):
    """
    Use the Newton-Raphson method to find the coupon rate that makes the present value of a bond equal to a target value.
    """
    ytm = coupon_rate
    ytm_old = 0
    for i in range(max_iterations):
        if abs(ytm - ytm_old) < tolerance:
            return (ytm, i)
        ytm_old = ytm
        pv = pv_bond_ytm(coupon_rate, ytm, n_maturity, face_value)
        ytm = ytm_old - (bond_price - pv) / derivative_func(bond_price, coupon_rate, ytm, n_maturity, face_value, *args, **kwargs)
    raise ValueError(f"Newton-Raphson method did not converge; {ytm} vs. {ytm_old} after {i} iterations.")

target_pv = 980
yield_to_maturity(target_pv, 4.32, 10, 1000), yield_to_maturity(1000, 4.32, 10, 1000)


((np.float64(4.571325175060531), 782), (np.float64(4.32), 1))

In [119]:
face_value = 1000
coupon_rate = 4.32
n_maturity = 10
market_prices = np.arange(800, 1200, 10)
max_iterations = 10000

analytic_ytm = np.array([yield_to_maturity(mv, coupon_rate, n_maturity, face_value) for mv in market_prices])
forward_diff_ytm = np.array([yield_to_maturity(mv, coupon_rate, n_maturity, face_value, forward_difference, max_iterations=max_iterations) for mv in market_prices])
backward_diff_ytm = np.array([yield_to_maturity(mv, coupon_rate, n_maturity, face_value, backward_difference, max_iterations=max_iterations) for mv in market_prices])
central_diff_ytm = np.array([yield_to_maturity(mv, coupon_rate, n_maturity, face_value, central_difference, max_iterations=max_iterations) for mv in market_prices])

In [120]:
analytic_ytm[-5:], forward_diff_ytm[-5:], backward_diff_ytm[-5:], central_diff_ytm[-5:]

(array([[  2.60644324, 963.        ],
        [  2.5017006 , 969.        ],
        [  2.3980147 , 974.        ],
        [  2.29536532, 978.        ],
        [  2.19373085, 983.        ]]),
 array([[  2.6064432 , 963.        ],
        [  2.50170055, 969.        ],
        [  2.39801465, 974.        ],
        [  2.29536528, 978.        ],
        [  2.19373081, 983.        ]]),
 array([[  2.60644329, 963.        ],
        [  2.50170065, 969.        ],
        [  2.39801474, 974.        ],
        [  2.29536537, 978.        ],
        [  2.1937309 , 983.        ]]),
 array([[  2.60644324, 963.        ],
        [  2.5017006 , 969.        ],
        [  2.3980147 , 974.        ],
        [  2.29536532, 978.        ],
        [  2.19373085, 983.        ]]))

In [121]:
def rmse(a, b):
    """
    Compute the root mean square error between two arrays.
    """
    return np.sqrt(np.mean((a - b) ** 2))

[rmse(analytic_ytm[:,0], i[:,0]) for i in [forward_diff_ytm, backward_diff_ytm, central_diff_ytm]]

[np.float64(4.3417213479657946e-08),
 np.float64(1.5522447024853424e-07),
 np.float64(1.5028024874488302e-12)]